<a href="https://colab.research.google.com/github/KarkiAnuj17/Automatic-text-summarization/blob/main/Automatic_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/dataset.zip", 'r')
zip_ref.extractall("/content/dataset")
zip_ref.close()

In [5]:
path="/content/dataset/dataset/test.csv"
df=pd.read_csv(path)
print(df.columns)
print(df.shape)

Index(['id', 'article', 'highlights'], dtype='object')
(11490, 3)


In [6]:
import spacy
tqdm.pandas()
nlp = spacy.load('en_core_web_sm')

In [7]:
def process_single_doc(doc):
    try:
        sentences = [sent.text for sent in doc.sents]  # Sentence splitting
        lemmatized_tokens = []
        for sent in doc.sents:
            tokens = [token.text for token in sent]  # Tokenization
            tokens_no_stop = [token.lemma_ for token in sent if not token.is_stop]  # Remove stop words and lemmatize
            lemmatized_tokens.append(tokens_no_stop)
        return {'sentences': sentences, 'tokens': tokens, 'tokens_no_stop': lemmatized_tokens}
    except Exception as e:
        print(f"Error processing document: {e}")
        return {'sentences': [], 'tokens': [], 'tokens_no_stop': []}
def process_docs_with_progress(docs):
    processed_docs = []
    for doc in tqdm(docs, desc='Processing Documents', total=len(df)):
        processed_docs.append(process_single_doc(doc))
    return processed_docs

In [8]:
# Use nlp.pipe to process documents in batches with reduced batch_size and n_process
docs_generator = nlp.pipe(df['article'], batch_size=20, n_process=2)

# Convert the generator to a list and process with progress bar
processed_data = process_docs_with_progress(docs_generator)

# Extract sentences, tokens, and tokens_no_stop from processed_data
df['sentences'] = [item['sentences'] for item in processed_data]
df['tokens'] = [item['tokens'] for item in processed_data]
df['tokens_no_stop'] = [item['tokens_no_stop'] for item in processed_data]

# Show processed data
print(df[['article','tokens_no_stop']].head())

Processing Documents: 100%|██████████| 11490/11490 [19:49<00:00,  9.66it/s]


                                             article  \
0  Ever noticed how plane seats appear to be gett...   
1  A drunk teenage boy had to be rescued by secur...   
2  Dougie Freedman is on the verge of agreeing a ...   
3  Liverpool target Neto is also wanted by PSG an...   
4  Bruce Jenner will break his silence in a two-h...   

                                           sentences  \
0  [Ever noticed how plane seats appear to be get...   
1  [A drunk teenage boy had to be rescued by secu...   
2  [Dougie Freedman is on the verge of agreeing a...   
3  [Liverpool target Neto is also wanted by PSG a...   
4  [Bruce Jenner will break his silence in a two-...   

                                              tokens  \
0  [British, Airways, has, a, seat, pitch, of, 31...   
1  [Last, year, a, 20, -, year, -, old, man, was,...   
2  [That, has, not, prevented, Forest, 's, owners...   
3  [Neto, joined, Fiorentina, from,  , Atletico, ...   
4  [In, February, ,, she, presented, ', A, Nat